In [ ]:
if 'google.colab' in str(get_ipython()):
    !pip install git+https://github.com/FullControlXYZ/fullcontrol --quiet
import fullcontrol as fc
from math import pi, tau, sin, cos, acos

In [ ]:
def map(v, il, ih, ol, oh):
    return (v - il) / (ih - il) * (oh - ol) + ol

class Printer:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.z = 0
        self.z_hop = 0.4 # mm
        self.z_speed = 30 # mm/s
        self.retract_length = 0.5 # mm
        self.retract_speed = 30 # mm/s
        self.extrude_width = 0.7 # mm
        self.extrude_height = 0.3 # mm
        self.extruding = False
        self.steps = []
        self.spacing = self.extrude_width
        self.update_geom()
        self.steps.append(fc.ManualGcode(text='SET_PRESSURE_ADVANCE ADVANCE=0.028'))

    def update_geom(self):
        self.steps.append(fc.ExtrusionGeometry(area_model='stadium', width=self.extrude_width, height=self.extrude_height))
        self.spacing = self.extrude_width - self.extrude_height * (1 - pi * 0.25)

    def set_layer(self, i):
        self.z = (i+1) * self.extrude_height
        self.steps.append(fc.Point(z=self.z))

    def move(self, x=None, y=None, z=None, hop=True):
        if self.extruding:
            self.extruding = False
            self.steps.append(fc.Extruder(on=False))
            #self.steps.append(fc.Point(e=-self.retract_length, f=self.rectract_speed*60))
            self.steps.append(fc.ManualGcode(text='G0 E{} F{}'.format(-self.retract_length, self.retract_speed*60)))
            if hop:
                #self.steps.append(fc.Point(z=self.z + self.z_hop, f=self.z_speed*60))
                self.steps.append(fc.ManualGcode(text='G0 Z{} F{}'.format(self.z + self.z_hop, self.z_speed*60)))
        self.steps.append(fc.Point(x=x, y=y, z=z))

    def extrude(self, x=None, y=None, z=None):
        if not self.extruding:
            self.extruding = True
            #self.steps.append(fc.Point(z=self.z, f=self.z_speed*60))
            self.steps.append(fc.ManualGcode(text='G0 Z{} F{}'.format(self.z, self.z_speed*60)))
            #self.steps.append(fc.Point(e=self.retract_length, f=self.rectract_speed*60))
            self.steps.append(fc.ManualGcode(text='G0 E{} F{}'.format(self.retract_length, self.retract_speed*60)))
            self.steps.append(fc.Extruder(on=True))
        self.steps.append(fc.Point(x=x, y=y, z=z))

def disc(p, center, diam, infill_angle, walls=1, seg=90):
    for w in range(walls):
        wall_d = diam - p.extrude_width/2 - (w*p.extrude_width)
        p.move(x=center[0], y=center[1] + wall_d / 2, hop=w==0)
        for i in range(seg):
            a = (i + 1) * tau / seg
            p.extrude(x=center[0] + sin(a) * wall_d / 2, y=center[1] + cos(a) * wall_d / 2)

    inner_diam = diam - p.extrude_width/2 - (walls*p.extrude_width)
    x = -inner_diam / 2 + p.extrude_width
    fwd_dir = True
    while x <= inner_diam / 2 - p.extrude_width:
        off = p.spacing / 2 * (-1 if x <= 0 else 1)
        y = inner_diam / 2 * sin(acos(map(x + off, -inner_diam / 2, inner_diam / 2, -1, 1))) - p.spacing / 2

        pos0 = (center[0] + x*cos(infill_angle) - y*sin(infill_angle), center[1] + y*cos(infill_angle) + x*sin(infill_angle))
        pos1 = (center[0] + x*cos(infill_angle) + y*sin(infill_angle), center[1] - y*cos(infill_angle) + x*sin(infill_angle))
        p.move(*(pos0 if fwd_dir else pos1), hop=False)
        p.extrude(*(pos1 if fwd_dir else pos0))

        fwd_dir = not fwd_dir
        x += p.spacing


In [ ]:
p = Printer()
p.steps.append(fc.Printer(print_speed=60 * 60, travel_speed=250 * 60))
p.steps.append(fc.Fan(speed_percent=100))

p.move(x=10, y=10, z=p.extrude_height)

center = (40, 40)
diam = 30

p.set_layer(0)
disc(p, center, diam, 0/360*tau, 3)
p.set_layer(1)
disc(p, center, diam, 60/360*tau, 1)
p.set_layer(2)
disc(p, center, diam, 120/360*tau, 3)

# slow down to give layers time to cool
p.steps.append(fc.Printer(print_speed=20 * 60))

center_diam = diam - p.spacing / 2
for layer in range(3, 50):
    p.set_layer(layer)
    start_off = (p.spacing / 2, center_diam/2 * sin(acos(p.spacing / center_diam)))

    p.move(x=center[0]+start_off[0], y=center[1]+start_off[1]-diam/4)
    p.extrude(x=center[0]+start_off[0], y=center[1]+start_off[1])

    segments = 90
    for i in range(1,segments):
        a = i * tau / segments
        p.extrude(x=center[0] + sin(a) * center_diam / 2, y=center[1] + cos(a) * center_diam / 2)

    p.extrude(x=center[0]-start_off[0], y=center[1]+start_off[1])
    p.extrude(x=center[0]-start_off[0], y=center[1]+start_off[1]-diam/4)

    start_pos_on_perimeter = ()

# End:
p.move(z=p.z+20)
p.steps.append(fc.Hotend(temp=0, wait=False))
p.steps.append(fc.Fan(speed_percent=0))

fc.transform(p.steps, 'plot')

In [ ]:
print_settings = {
}
start_steps = [
    fc.Extruder(relative_gcode=True),
    fc.ManualGcode(text='PRINT_START EXTRUDER=215 BED=60'),
]
end_steps = [fc.ManualGcode(text='PRINT_END')]
fc.transform(start_steps + p.steps + end_steps, 'gcode', fc.GcodeControls(printer_name='custom', save_as='seam-test', initialization_data=print_settings))